# 알고리즘 트레이딩 실습 2

<a href="https://colab.research.google.com/github/SLCFLAB/Fintech2022/blob/main/AT_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 실습 1 복습

In [ ]:
!pip install -U finance-datareader

In [ ]:
!pip install -U cvxopt

In [ ]:
import FinanceDataReader as fdr
print(fdr.__version__)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxopt as opt
from cvxopt import solvers

### To-Do

다음의 조건에 해당하는 각 지수별 price data를 pandas의 DataFrame 형태로 생성하고 a라는 이름으로 선언하시오.      
https://financedata.github.io/posts/finance-data-reader-users-guide.html
* 조건 1: 각 row의 인덱스는 날짜, 각 컬럼은 KOSPI200 지수, S&P500 지수, 나스닥 지수, 닛케이225 선물, USD/KRW로 구성.
* 조건 2: 종가 기준, 기간은 2010년 이후 현재까지.

### Efficient Frontier 구하기

In [ ]:
holding_periods=1
n_term=252 # dailiy return을 annualize
data = a

returns = data.pct_change(holding_periods).iloc[holding_periods:] # holding period return vector
exp_rets = returns.mean() * n_term # average return -- > APR(annual percentage rate)
cov_mat = returns.cov() * n_term # covariance
corr_mat = returns.corr() # corr

In [ ]:
plt.matshow(corr_mat)
plt.show()

In [ ]:
import seaborn as sns

sns.heatmap(corr_mat)

Mean Variance Portfolio를 구성

In [ ]:
def mean_var_portfolio(cov_mat, exp_rets, target_ret):
    
    n = len(cov_mat)

    # Objective function
    # min (1/2)*w.T*P*w
    P = opt.matrix(cov_mat.values) 
    q = opt.matrix(0.0, (n, 1))


    # Constraints Gw <= h
    # exp_rets*w >= target_ret and w >= 0
    G = opt.matrix(np.vstack((-exp_rets.values, -np.identity(n))))
    h = opt.matrix(np.vstack((-target_ret, np.zeros((n, 1)))))

    # Constraints Aw = b
    # sum(w) = 1
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Solve
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)

    # Put weights into a labeled series
    w = pd.Series(sol['x'], index=cov_mat.index)

    return w

In [ ]:
max_risk=0.5
max_ret = exp_rets.max()
min_ret = exp_rets[exp_rets > 0].min()

n_step = 100
step_diff = (max_ret - min_ret)/n_step

risks = []
rets = []

for step in range(0, n_step+1):
    target_ret = min_ret + (step * step_diff) # y축을 아래부터 시작해서 위로 조금씩 올려가면서
    _w = mean_var_portfolio(cov_mat, exp_rets, target_ret) # 최적화를 돌려서 나온 _w임.
    risk = np.sqrt(np.dot(np.dot(_w, cov_mat), _w))

    if risk > max_risk:
        break

    risks.append(risk)
    ret = np.dot(exp_rets, _w)
    rets.append(ret)

In [ ]:
plt.plot(risks, rets)
plt.show()

In [ ]:
plt.plot(risks, rets)

var = np.diag(cov_mat)
plt.scatter(np.sqrt(var), exp_rets)

plt.show()

### To-do

개미투자자 A는 인덱스에 투자하는 것보다 개별 종목에 투자하는 것을 선호한다. 한국 주식보다는 미국 주식에 투자하는 것을 선호하는 A는 평소 Warren Buffett을 굉장히 존경해왔다고 한다. 최근 뉴스를 보고 Buffett이 관심을 가졌던 주식들을 파악한 A는 다음과 같은 회사들로 포트폴리오를 구성하려 한다.
* Apple
* Occidental Petroleum
* Coca Cola
* Exxon Mobil
* Microsoft

하지만 주변 사람들의 조언을 들어본 결과, 주식으로만 포트폴리오를 구성하는 것은 위험하다고 판단이 든 A는 자신의 포트폴리오에 금(선물)을 포함하기로 결정했다. 총 6개의 종목으로 포트폴리오를 구성한 A에게 도움을 주기 위한 Efficient Frontier를 그려보자

위의 조건에 해당하는 종목들에 대하여 2016년 이후 종가기준으로 dataframe을 생성하고 Efficient Frontier를 그려보시오.

## 최소 분산 포트폴리오
![](https://ift.world/wp-content/uploads/2018/06/wsi-imageoptim-1-1.png)

**최소 분산 포트폴리오** : 자산들로 구성되는 포트폴리오 중 최소 분산을 갖는 포트폴리오.    
Efficient Frontier 상에서 분산이 가장 작은 점이 된다.

### To-Do

#### A. 앞서 구한 Efficient Frontier를 바탕으로, Efficient frontier 상에서 variance가 가장 작은 점을 구하시오. 

In [ ]:
# Efficient Frontier

max_risk=0.5
max_ret = exp_rets.max()
min_ret = exp_rets[exp_rets > 0].min()

n_step = 1000
step_diff = (max_ret - min_ret)/n_step

risks = []
rets = []
weights = []

for step in range(n_step):
    target_ret = min_ret + (step * step_diff) # y축을 아래부터 시작해서 위로 조금씩 올려가면서
    _w = mean_var_portfolio(cov_mat, exp_rets, target_ret) # 최적화를 돌려서 나온 _w임.
    risk = np.sqrt(np.dot(np.dot(_w, cov_mat), _w))

    if risk > max_risk:
        break
    
    risks.append(risk)
    ret = np.dot(exp_rets, _w)
    rets.append(ret)
    weights.append(_w)

min_risk_idx = ??

#### B. 구한 minimum variance portfolio의 risk, return, portfolio weight를 프린트하시오.

In [ ]:
min_risk_idx = ??
min_risk = ??
min_risk_ret = ??
min_risk_weight = ??
print("Risk of minimum variance portfolio : ", min_risk)
print("Return of minimum variance portfolio : ", min_risk_ret)
print("Weight of minimum variance portfolio : ")
print(min_risk_weight)

#### C. Efficient frontier와 Minimum variance portfolio를 plotting하시오.

## 탄젠시 포트폴리오(Tangency Portfolio)

![](https://www.researchgate.net/profile/Teodosii_Rachev/publication/24071755/figure/fig2/AS:667097013161996@1536059730503/The-Efficient-frontier-and-the-tangent-portfolio.png)

앞서서는 주어진 자산들로 구성되는 포트폴리오들을 분석하여 efficient frontier를 구하고, minimum variance portfolio를 구하는 방법을 살펴보았다.     
이제 앞선 구성자산들에 더하여, risk-free asset이 존재할 시 어떤 portfolio들을 만들 수 있는 지 알아보자.
* Risk free asset : 한국의 경우 국고채 3년 금리 사용 --> 1%
* Efficient frontier 상의 포트폴리오 중, Sharpe ratio가 가장 큰 포트폴리오 --> Tangency portfolio

### To-Do

#### 앞서 구한 Efficient Frontier를 바탕으로, Efficient frontier 상에서 Sharpe ratio가 가장 커지는 tangency portfolio 점을 구하시오.

In [ ]:
# Efficient Frontier

max_risk=0.5
max_ret = exp_rets.max()
min_ret = exp_rets[exp_rets > 0].min()

n_step = 100
step_diff = (max_ret - min_ret)/n_step

risks = []
rets = []
weights = []

for step in range(n_step):
    target_ret = min_ret + (step * step_diff) # y축을 아래부터 시작해서 위로 조금씩 올려가면서
    _w = mean_var_portfolio(cov_mat, exp_rets, target_ret) # 최적화를 돌려서 나온 _w임.
    risk = np.sqrt(np.dot(np.dot(_w, cov_mat), _w))

    if risk > max_risk:
        break
    
    risks.append(risk)
    ret = np.dot(exp_rets, _w)
    rets.append(ret)
    weights.append(_w)


rf_rate = 0.01

slopes = []

# ToDo
### slopes list에 기울기 추가
### max_tangency_idx, max_tangency 구하기

#### a) 구한 tangency portfolio의 risk, return, sharpe ratio, portfolio weight를 프린트하시오.

In [ ]:
max_tangency_risk = ??
max_tangency_return = ??
max_tangency_weight = ??

#### b) tangency portfolio와 risk-free rate를 연결한 직선과, efficint frontier를 plotting 하시오

## 백테스팅(Backtesting)

우리가 세운 전략을 **실제로 historical data에 적용**하면 어떻게 되는가?    
먼저 생각해야 할 것.
1) 이전 몇 달 치의 데이터를 사용하여 Tangency portfolio weight를 구성할 것인가? (Lookback period)    
2) **리밸런싱**을 얼마의 주기로 할 것인가?

### 5개 asset으로 tangency portfolio를 구성하는 backtesting

* A) 리밸런싱 시점에서 tangency portfolio를 구성할 때, 이전 6달(120일)의 데이터를 바탕으로 만든다.
* B) 리밸런싱은 3달(60일) 1분기마다 진행한다.
* C) return과 covariance를 구할 때는 월평균으로 구하여 연평균 수익률로 변화시켜준다.
* D) 시작 자산은 10000원에서 시작하여, 각 날짜마다 portfolio value가 어떻게 되는 지를 구한다.

In [ ]:
data = a['2015-01-01':]

In [ ]:
def create_data(data):
    
    result = data.pct_change(20).iloc[20:]
    
    returns = result
    avg_rets = returns.mean()*12
    cov_mat = returns.cov()*12

    return returns, cov_mat, avg_rets

In [ ]:
def mean_var_portfolio(cov_mat, exp_rets, target_ret):
    
    n = len(cov_mat)

    # Objective function
    # min (1/2)*w.T*P*w
    P = opt.matrix(cov_mat.values) 
    q = opt.matrix(0.0, (n, 1))


    # Constraints Gw <= h
    # exp_rets*w >= target_ret and w >= 0
    G = opt.matrix(np.vstack((-exp_rets.values, -np.identity(n))))
    h = opt.matrix(np.vstack((-target_ret, np.zeros((n, 1)))))

    # Constraints Aw = b
    # sum(w) = 1
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Solve
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)

    # Put weights into a labeled series
    w = pd.Series(sol['x'], index=cov_mat.index)

    return w

In [ ]:
def tangency_portfolio(cov_mat, exp_rets, target_ret, min_rets, max_rets, rf_rate = 0.01):
    
    n = len(cov_mat)
    P = opt.matrix(cov_mat.values)
    q = opt.matrix(0.0, (n, 1))
    
    tangency_queue = []
    weights_queue = []
    return_queue = []
    cov_queue = []
    
    step = (max_rets - min_rets)/100
    for i in range(100):
                
        try:
        
            target_ret = min_rets + (i * step)

            # Constraints Gx <= h

            G = opt.matrix(np.vstack((-exp_rets.values, -np.identity(n))))
            h = opt.matrix(np.vstack((-target_ret, np.zeros((n, 1)))))

            # Constraints Ax = b
            # sum(x) = 1
            A = opt.matrix(1.0, (1, n))
            b = opt.matrix(1.0)

            solvers.options['show_progress'] = False
            sol = solvers.qp(P, q, G, h, A, b)

            if sol['status'] != 'optimal':
                pass

            # Put weights into a labeled series
            weights = pd.Series(sol['x'], index=cov_mat.index)
            
            tangency_ = (np.dot(exp_rets, weights) - rf_rate) / np.sqrt(np.dot(np.dot(weights, cov_mat), weights))
            tangency_queue.append(tangency_)
            weights_queue.append(weights)
            return_queue.append(np.dot(exp_rets, weights))
            cov_queue.append(np.sqrt(np.dot(np.dot(weights, cov_mat), weights)))
        
        except:
            pass

    
    tan_index = tangency_queue.index(max(tangency_queue))
    
    weights = weights_queue[tan_index]

    return weights

In [ ]:
data = data.dropna()

In [ ]:
ret = data.pct_change().iloc[1:]

In [ ]:
data.shape

In [ ]:
def back_test(data, ret):

    # Window_size는 tangency portfolio를 구할 때 이전 몇 달의 데이터를 사용할 것인지 나타낸다.
    total_days = data.shape[0]
    
    # Rebalancing을 한 달마다 진행하는 경우, 'M'으로 rebalance_period는 20이 된다.
    rebalance_period = 60 # 3개월에 한번 = 분기당 한번
    window_size = 120 # 지난 6개월

    returns, cov_mat, avg_rets= create_data(data)
    
    # n_rebalance : 총 몇 번의 리밸런싱을 하게 되는지?
    n_rebalance = (total_days - window_size) // rebalance_period
    w = []
    rebalancing_date=[]
    original_weights = []
    
    rebal = 60
    for i in range(window_size, total_days):
        # 리밸런싱을 할 때 마다 Tangency portfolio weight를 새로 구해야 한다.
        # 이때 리밸런싱 시점 이전의 총 120일 간의 데이터를 사용한다.
        
        if rebal % rebalance_period == 0:
            #print('rebalance')
            _returns, _cov_mat, _avg_rets= create_data(data[(i-window_size):i])
            rebalancing_date.append(data.index[i:i+1]) # rebalance_period 씩 늘려가자
            rebal = 0
            _target_ret = _avg_rets.quantile(0.7)
            _min_ret = _avg_rets.min()
            _max_ret = _avg_rets.max()
            _w = tangency_portfolio(_cov_mat, _avg_rets, _target_ret, _min_ret, _max_ret)
            print(n_rebalance)
            n_rebalance = n_rebalance - 1
            print(_w)
            _w.plot.bar()
            plt.show()
            plt.close()
            _w = np.asarray(_w.values) #np.array([0.0, 0.0, 1.0, 0.0, 0.0])
            w.append(_w)
            

        rebal = rebal + 1
        
    
    price = data
    price = np.array(price)
    
    port = [] # 여기에 날마다 변하는 Portfolio value를 저장한다.
    port_date = [] # 여기에 portfolio의 data를 저장한다.
    portfolio_value = 10000
    port_each_money = [] # 그 날의 portfolio의 구성이다. Ex) 만약 Asset1에 6000원, Asset2에 4000원, 나머지에 0원이 있다면 [6000, 4000, 0, 0, 0]이 된다.
    
    rebal = 60
    j = 0
    for i in range(window_size, total_days):
        
        if i == window_size: # 초기화
            port_each_money = portfolio_value * w[j]
            rebal = 1
            # port_each_money 값을 다 더한 게 그 날의 portfolio_value가 된다.
            portfolio_value = port_each_money.sum()
            # port에 portfolio_value를 저장하여 준다.
            port.append(portfolio_value)
            j = j+1


        else:
            #각 날마다 자산의 가격이 바뀌므로, port_each_money 변화한다.
            port_each_money = port_each_money * (1+ret.iloc[i])
            portfolio_value = port_each_money.sum()
            if rebal % rebalance_period == 0:
                rebal = 0
                # 리밸런싱을 하는 날이다. Portfolio value를 새로운 tangency portfolio weight로 맞춰준다.
                port_each_money = portfolio_value * w[j]
                # port_weight의 값을 다 더한 게 그 날의 portfolio_value가 된다.
                portfolio_value = port_each_money.sum()
                # port에 portfolio_value를 저장하여 준다.
                port.append(portfolio_value)
                j = j+1
            
            else:
                # 리밸런싱 날짜가 아닌 평범함 날짜. 
                # port_each_money 값을 다 더한 게 그 날의 portfolio_value가 된다.
                portfolio_value = port_each_money.sum()
                # port에 portfolio_value를 저장하여 준다.
                port.append(portfolio_value)
            
        rebal = rebal + 1
        port_date.append(data.index[i:i+1][0])
    
    portfolio = pd.DataFrame(port, columns = ['portfolio_value'], index = port_date)
    
    return w, portfolio, rebalancing_date

In [ ]:
data

In [ ]:
ret

In [ ]:
w, portfolio, rebalancing_date = back_test(data[1:], ret)

In [ ]:
portfolio.plot()

In [ ]:
# Backtest Result Plotting
plt.plot(portfolio, label="portfolio")

for symbol in ['KS200', 'US500', 'IXIC', 'JP225', 'USD/KRW']:
    plt.plot(data[121:][symbol] * 10000 / data[121:][symbol][0], label='{}'.format(symbol))

plt.legend()
plt.show()

### To-Do

#### 앞서 구한 리밸런싱 포트폴리오의 Sharpe ratio를 plot 하시오.

## Backtest with A's portfolio

다른 포트폴리오 유니버스에 대한 백테스팅을 진행해보자. 앞서 생성한 투자자 A의 데이터프레임을 바탕으로 동일한 전략에 대한 백테스팅을 진행해보자. 최종적인 시간에 따른 수익 변화를 plot하시오.

Final Value of Portfolio

### To-Do

더 좋은 전략이 있을까요? 더 좋은 전략을 구상하시고 백테스팅을 진행해보세요!